# ER graphs

Ces exercices portent principalement sur les 2 premiers exercices vus en classe lors de la dernière séance. 

1-Generate 10 replicas of ER graphs with N=1000 nodes and connection probabilities p=[0.00005, 0.0001, 0.0002, 0.001, 0.002,0.005, 0.01,0.015, 0.02]

2- show visually and with a linear fit that $\langle k\rangle=Np$

3- Are the networks connected? Which is the size of the giant component?  Plot size of the giant component as a function of p. What do you observe?

<mark>10 réplications de graphe avec 1000 nœuds avec REECOUTER Montrer visuellement ces relation. 
2. Très important : concerne la connectivité du réseau. L’idée est que pour chaque valeur de p, on doit regarder la taille moyenne de la composante maximale. Et on nous laisse donner les conclusions là-dessus.</mark>


# COMPARISON WITH REAL NETWORKS

1- Take the twitch networks from last lesson. Calculate their average clustering coefficient. Calculate the average clustering coefficient for several replicas of a random graph with the same number of nodes and the same number of edges (using p=2E/N(N-1) ) of the twich networks. Compare the real values and the random values. What do you observe? How can you interpret this?

OPTIONAL: perform a significance test to see if the value observed in real networks is equal to the value observed in the random case. 

2- OPTIONAL A better way to randomize network structures consists in rewiring the links preserving the degree structure:

(1) take two links (n1,n2) and (n3,n4). 

(2)exchange one of the extremes in order to have (n1,n4) and (n3,n2)

(3) Iterate this procedure for a number of times proportional to the number of edges

Write the script performing this procedure and apply this to one of the Twitch networks. 




<mark>On cherche à utiliser des réseaux aléatoires. On regarde les réseaux twitch de la dernière fois (France, Portugal, les deux ensemble comme on veut). On calcule l’erreur clustering moyenne et ensite l’average clustering pour un graphe aléatoire qui a le même nombre de nœuds et de liens et on fait la comparaison entre les valeurs obtenus du clustering pour le réseau réel et réseau aléatoire. Si on sait le faire, on peut faire si on veut un test (significativité) pour voir comment comparer ces valeurs (mais ça c’est pas obligatoire). 
Dernière chose : algorithme qui correspond à une possibilité de changer structure des réseau sans changer [réécouter] et réaliser cet algorithme sur le réseau twitch</mark>